In [ ]:
import numpy as np
import numba as nb
from numba import cuda
import time

LIMIT = 13**6

In [ ]:

def isprime(n:int):
      if n == 2: return True
      if n % 2 == 0: return False
      return all(n % i != 0 for i in range(3, int(n**0.5)+1, 2))

def primesInRange(limit:int)->list[int]:
      return [isprime(x) for x in range(limit)]

In [ ]:

#Porównywanie pythona nawet nie miało sensu, bo wykresy były kompletnie nieczytelne
# start = time.time()
# primesInRange(LIMIT)
# python = time.time() - start

In [ ]:
%%script pypy3 --out pypy
import time
LIMIT = 13**6
def isprime(n):
      if n == 2: return True
      if n % 2 == 0: return False
      return all(n % i != 0 for i in range(3, int(n**0.5)+1, 2))

def primesInRange(limit):
      return [isprime(x) for x in range(limit)]

start= time.time()
pypy = primesInRange(LIMIT)
end = time.time() - start
print(end)

In [ ]:
pypy = float(pypy)

In [ ]:

@nb.jit(nopython=True,parallel = True,fastmath = True)
def numba_isprime(n):  # sourcery skip: invert-any-all, use-any, use-next
      if n == 2: return True
      if n % 2 == 0: return False
      for i in range(3, int(n**0.5)+1, 2):
            if n%i==0:
                  return False
      return True
@nb.jit(nopython=True,parallel= True)
def numba_primesInRange(limit):
      arr = [False for _ in range(limit)]
      for id in range(limit):
            arr[id] = numba_isprime(id)
      return arr


In [ ]:
start = time.time()
tmp = numba_primesInRange(LIMIT)
numba_jit = time.time() - start

In [14]:

@cuda.jit
def numba_isprime(n):  # sourcery skip: invert-any-all, use-any, use-next
      if n == 2: return True
      if n % 2 == 0: return False
      for i in range(3, int(n**0.5)+1, 2):
            if n%i==0:
                  return False
      return True
@cuda.jit
def numba_cuda_primesInRange(arr):
    for index in range(cuda.grid(1),len(arr),cuda.gridsize()):
        arr[index] = numba_isprime(index)
    

threadsperblock = 32
blockspergrid = (LIMIT + (threadsperblock - 1)) // threadsperblock
arr = np.array(range(LIMIT),np.bool8)
arr = cuda.to_device(arr)


In [15]:
start = time.time()
numba_cuda_primesInRange[blockspergrid, threadsperblock](arr)
cuda.synchronize()
numbda_cuda = time.time()- start


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
No implementation of function Function(<class 'numba.cuda.stubs.gridsize'>) found for signature:
 
 >>> gridsize()
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'gridsize': File: numba/cuda/cudadecl.py: Line 20.
    With argument(s): '()':
   Rejected as the implementation raised a specific error:
     TypingError: missing a required argument: 'ndim'
  raised from /home/mkwarta/Studia/PR/Hist_equalization/venv/lib/python3.10/site-packages/numba/core/typing/templates.py:412

During: resolving callee type: Function(<class 'numba.cuda.stubs.gridsize'>)
During: typing of call at /tmp/ipykernel_6127/2750909971.py (11)


File "../../../../../tmp/ipykernel_6127/2750909971.py", line 11:
<source missing, REPL/exec in use?>


In [ ]:
labels = ['pypy','numba_jit','cuda']
values = list(map(lambda x:round(x,2),[pypy,numba_jit,numbda_cuda]))
import matplotlib.pyplot as plt
plt.grid(True)
plt.ylim(0,60)
plt.bar(labels,values)
print(*list(zip(labels,values)),sep='\n')